In [1]:
import os
import requests
import base64
import json
from jinja2 import Environment, FileSystemLoader

def page_link(a):
    if ' ' in a:
        return a.replace(" ", "_")
    else: 
        return a

# Set environment variables
token = os.environ['LAL']
url_base = "https://api.github.com/repos/harshulgupt/json_pvt/contents/jsons/"
headers = {
    "Authorization": "Token " + token
}

# Fetch and decode JSON files
json_files = ["general", "research"]
data = {}
for json_file in json_files:
    response = requests.get(url_base + json_file + ".json", headers=headers)
    content = response.json()["content"]
    decoded = base64.b64decode(content).decode()
    data[json_file] = json.loads(decoded)

# Create instance of Environment and render templates
environment = Environment(loader=FileSystemLoader('templates/'))
environment.globals['page_link'] = page_link
template_no_twitter = environment.get_template('research_page_no_twitter.html.j2')
template_twitter = environment.get_template('research_page_twitter.html.j2')

# Render content and write to file
for i,project in enumerate(data["research"]):
    filename = f"sub_research/{ page_link(data['general']['Research'][i]) }.html"
    if project["twitter"] == "None":
        content = template_no_twitter.render(general=data["general"], research=project)
    else:
        content = template_twitter.render(general=data["general"], research=project)
    with open(filename, mode="w", encoding="utf-8") as message:
        message.write(content)
